# Phase 1 –*MLOps Course Project: Online News Popularity Dataset*
### Activity 03: Model Construction, Tuning, and Evaluation
**Team:** *41*  
**Course:** *Machine Learning Operations (MLOps)*  
**Institution:** Tecnológico de Monterrey  
**Date:** *October 12th, 2025*  

---

### **Team Members & Roles**
- **Data Engineer:** *Angel Iván Ahumada Arguelles*
- **Data Scientist:** *Steven Sebastian Brutscher Cortez*  
- **Software Engineer:** *Ana Karen Estupiñán Pacheco*  
- **ML Engineer:** *Felipe de Jesús Gutiérrez Dávila*  

---

> *This notebook corresponds to the third collaborative deliverable of the MLOps course project.  
**Purpose:** Train baseline and tuned models using the preprocessed dataset, evaluate performance with appropriate metrics, and log decisions and results.*

**Primary author:** Felipe — ML Engineer  
**Collaborators:** Steven (Data Scientist); Ana Karen (Software Engineer); Ángel (Data Engineer)

# 03. Model Construction, Tuning, and Evaluation

## Objective
This notebook focuses on building, training, and evaluating machine learning models to predict the popularity of online news articles measured by the number of `shares`.

Following the completion of the previous notebook (*02_Data_Exploration_and_Preprocessing*),
we now start the final step of Phase 1 of the MLOps project.

The preprocessed dataset `df_ready_for_modeling.csv` includes 60 standardized numeric features and one target variable (`shares`). These features are clean, normalized, and free from invalid values, allowing us to focus entirely on model development and performance evaluation.

## Workflow Overview
1. **Data loading and verification**  
   Import the preprocessed dataset and confirm feature integrity.

2. **Data splitting**  
   Divide the dataset into training and testing sets for fair evaluation.

3. **Model construction**  
   Implement several baseline models to predict `shares`:
   - Linear Regression  
   - K-Nearest Neighbors (KNN)  
   - Random Forest Regressor  

4. **Model tuning and evaluation**  
   Perform cross-validation and grid search for hyperparameter optimization.
   Compare models using standard regression metrics (RMSE, MAE, R²).

5. **Model persistence and documentation**  
   Save trained models and performance metrics for reproducibility
   and integration with the DVC versioning workflow.

## Expected Output
By the end of this notebook, we will obtain:
- A trained and validated regression pipeline ready for deployment.
- Performance metrics for multiple algorithms.
- Serialized model artifacts for DVC tracking.

This notebook corresponds to *Phase 1.3 – Model Construction, Tuning, and Evaluation*.

In [ ]:
# ================================================================
# 1. Setup and Data Loading
# ================================================================

# --- 1.1. Import core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1.2. Machine Learning utilities
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib  # for model persistence

# --- 1.3. Mount Google Drive (if using Google Colab)
from google.colab import drive
drive.mount('/content/drive')

# --- 1.4. Define dataset path
path_ready = "/content/drive/MyDrive/Colab Notebooks/Maestría MNA - IA Aplicada/TR5 - MLOPS/Fase 1/final_dataset/df_ready_for_modeling.csv"

# --- 1.5. Load dataset
df = pd.read_csv(path_ready)

# --- 1.6. Quick inspection
print("Dataset successfully loaded.")
print(f"Shape: {df.shape}")
print("\nPreview:")
display(df.head())

# --- 1.7. Basic statistics
print("\nDescriptive statistics (target variable 'shares'):")
display(df["shares"].describe())

# --- 1.8. Feature-target separation
X = df.drop(columns=["shares"])
y = df["shares"]

# --- 1.9. Data split for model training/testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nTrain shape: {X_train.shape}")
print(f"Test shape:  {X_test.shape}")

Mounted at /content/drive
Dataset successfully loaded.
Shape: (39235, 61)

Preview:


,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,mixed_type_col,shares
0,1.780195,0.713746,-0.776915,0.895116,0.173457,0.893557,-0.715545,-0.413284,-0.495268,-0.412623,...,-0.234578,-0.719130,-0.275450,0.078881,0.689280,-0.979120,-1.836273,-0.002221,-0.009950,593.0
1,1.780195,-0.670630,-0.689217,0.484749,0.173457,0.742778,-0.822698,-0.761478,-0.495268,-0.412623,...,-0.234578,1.112537,1.381404,0.078881,-0.867998,-0.265225,0.820537,-0.041180,0.595337,711.0
2,1.780195,-0.670630,-0.796404,0.278255,0.173457,-0.081172,-0.822698,-0.761478,-0.495268,-0.412623,...,0.989082,-1.643214,-0.973074,-0.272991,-0.867998,-0.265225,0.820537,-0.041180,-0.009950,1500.0
3,1.780195,-0.670630,-0.016862,-0.219206,0.173457,-0.069791,-0.179783,-1.109672,-0.495268,-0.412623,...,0.173309,-0.875144,-0.275450,-0.624863,-0.867998,-0.265225,0.820537,-0.041180,0.798481,1200.0
4,1.780195,1.175205,1.301052,-0.833817,0.173457,-0.872282,0.891741,3.361984,2.357859,-0.412623,...,0.989082,0.309041,0.073361,0.606689,0.547710,0.253972,-1.594745,-0.012846,0.346589,505.0



Descriptive statistics (target variable 'shares'):


,shares
count,39235.000000
mean,2905.619181
std,4065.543844
min,1.000000
25%,957.000000
50%,1400.000000
75%,2800.000000
max,21513.906872



Train shape: (31388, 60)
Test shape:  (7847, 60)


## 2. Feature Selection and Pruning

Before training any model, it is crucial to ensure that the input features are independent, relevant, and non-redundant. Based on correlation analysis and prior domain knowledge, we identified several columns that are either:

- **Highly correlated** with others (creating redundancy).  
- **Derived or averaged metrics** that can be reconstructed from base variables.  
- **Unavailable at prediction time** (data leakage risk).  
- **Non-predictive identifiers** or purely temporal variables.

The following columns will be removed from the dataset:

| Category | Columns | Rationale |
|-----------|----------|-----------|
| Derived averages | `average_token_length`, `kw_avg_min`, `kw_avg_max`, `kw_avg_avg`, `avg_positive_polarity`, `avg_negative_polarity` | Duplicated information already captured by related variables. |
| Data leakage | `self_reference_min_shares`, `self_reference_max_shares`, `self_reference_avg_sharess` | Depend on post-publication performance (unavailable during prediction). |
| Temporal / identifier | `timedelta`, `url` | Non-predictive or already removed. |
| Redundant binary flags | `is_weekend`, `weekday_is_sunday` | Redundant with weekday columns (Saturday/Sunday flags). |
| Mixed or invalid | `mixed_type_col` | Inconsistent data type, not suitable for modeling. |

This pruning step ensures the model learns from stable, meaningful, and non-redundant features.

In [ ]:
# ================================================================
# 2. Feature Selection and Pruning
# ================================================================

# --- 2.1. List of columns to drop
cols_to_drop = [
    "average_token_length", "kw_avg_min", "kw_avg_max", "kw_avg_avg",
    "avg_positive_polarity", "avg_negative_polarity",
    "self_reference_min_shares", "self_reference_max_shares", "self_reference_avg_sharess",
    "timedelta", "is_weekend", "weekday_is_sunday", "mixed_type_col"
]

# --- 2.2. Drop columns only if they exist in the dataset
existing_cols = [c for c in cols_to_drop if c in X.columns]
print(f"Columns found and removed ({len(existing_cols)}): {existing_cols}\n")

X = X.drop(columns=existing_cols, errors='ignore')

# --- 2.3. Check resulting shape and column count
print(f"Remaining features: {X.shape[1]}")
print(f"Train shape: {X_train.shape[0]} rows × {X.shape[1]} columns")
print("\nUpdated feature list preview:")
print(sorted(X.columns[:10]))

# --- 2.4. Optional: check remaining correlations above 0.85
corr_matrix = X.corr().abs()
high_corr_pairs = [(c1, c2, corr_matrix.loc[c1, c2])
                   for c1 in corr_matrix.columns for c2 in corr_matrix.columns
                   if c1 != c2 and corr_matrix.loc[c1, c2] > 0.85]

if high_corr_pairs:
    print("\nHighly correlated pairs (|r| > 0.85):")
    for c1, c2, corr in sorted(high_corr_pairs, key=lambda x: -x[2])[:10]:
        print(f"  {c1} ↔ {c2} → r = {corr:.3f}")
else:
    print("\nNo remaining correlations above 0.85 — dataset ready for modeling.")

Columns found and removed (13): ['average_token_length', 'kw_avg_min', 'kw_avg_max', 'kw_avg_avg', 'avg_positive_polarity', 'avg_negative_polarity', 'self_reference_min_shares', 'self_reference_max_shares', 'self_reference_avg_sharess', 'timedelta', 'is_weekend', 'weekday_is_sunday', 'mixed_type_col']

Remaining features: 47
Train shape: 31388 rows × 47 columns

Updated feature list preview:
['n_non_stop_unique_tokens', 'n_non_stop_words', 'n_tokens_content', 'n_tokens_title', 'n_unique_tokens', 'num_hrefs', 'num_imgs', 'num_keywords', 'num_self_hrefs', 'num_videos']

Highly correlated pairs (|r| > 0.85):
  n_unique_tokens ↔ n_non_stop_unique_tokens → r = 0.851
  n_non_stop_unique_tokens ↔ n_unique_tokens → r = 0.851


## 2.1 Interpretation of pruning results and next action

After removing clearly redundant, leaked, or non-predictive fields, we retained **47 features**.  
A quick multicollinearity scan (|r| > 0.85) surfaced one remaining high-correlation pair:

- `n_unique_tokens` ↔ `n_non_stop_unique_tokens` (r ≈ 0.851)

Both are proportions in [0, 1] and capture very similar information about lexical diversity.  
To reduce collinearity and keep models simpler and more stable, we will **drop `n_non_stop_unique_tokens`** and retain `n_unique_tokens` (it is the canonical, easier-to-interpret signal).

With the feature space finalized, we will train a fast set of baseline regressors and compare them under a consistent evaluation protocol. We will model **log(1 + shares)** to stabilize the heavy-tailed target, and report metrics back on the original scale.

In [ ]:
# ================================================================
# 3. Baselines: quick model sweep on log(1+shares)
# ================================================================

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

# --- 3.0. Drop the remaining highly correlated feature
if "n_non_stop_unique_tokens" in X.columns:
    X = X.drop(columns=["n_non_stop_unique_tokens"])
    X_train = X_train.drop(columns=["n_non_stop_unique_tokens"])
    X_test  = X_test.drop(columns=["n_non_stop_unique_tokens"])
print(f"Final feature count: {X.shape[1]}")

# --- 3.1. Target transform: log(1 + y)
y_train_log = np.log1p(y_train)
y_test_log  = np.log1p(y_test)

# --- 3.2. Helper: evaluate on original scale with inverse transform
def eval_regression(y_true, y_pred_log):
    """
    y_pred_log: predicción en escala log1p.
    Devuelve MAE, RMSE y R2 en la escala original de 'shares'.
    """
    y_pred = np.expm1(y_pred_log)
    mae  = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))  # <- sin 'squared=False'
    r2   = r2_score(y_true, y_pred)
    return mae, rmse, r2

# --- 3.3. Models
models = {
    "Linear": Pipeline([
        ("scaler", StandardScaler(with_mean=True, with_std=True)),
        ("reg", LinearRegression())
    ]),
    "Ridge(α=5)": Pipeline([
        ("scaler", StandardScaler()),
        ("reg", Ridge(alpha=5.0, random_state=42))
    ]),
    "KNN(k=15)": Pipeline([
        ("scaler", StandardScaler()),
        ("reg", KNeighborsRegressor(n_neighbors=15, weights="distance"))
    ]),
    "RandomForest": RandomForestRegressor(
        n_estimators=300, max_depth=None, min_samples_leaf=2,
        n_jobs=-1, random_state=42
    ),
}

# --- 3.4. Train/validation protocol
kf = KFold(n_splits=5, shuffle=True, random_state=42)

results = []
for name, model in models.items():
    # Fit on train (log scale)
    model.fit(X_train, y_train_log)

    # CV predictions on train folds (log scale) to gauge generalization
    y_train_log_cv = cross_val_predict(model, X_train, y_train_log, cv=kf, n_jobs=-1, method="predict")
    mae_tr, rmse_tr, r2_tr = eval_regression(y_train, y_train_log_cv)

    # Holdout test (log scale)
    y_test_log_pred = model.predict(X_test)
    mae_te, rmse_te, r2_te = eval_regression(y_test, y_test_log_pred)

    results.append({
        "model": name,
        "features": X.shape[1],
        "MAE_train": round(mae_tr, 2),
        "RMSE_train": round(rmse_tr, 2),
        "R2_train": round(r2_tr, 4),
        "MAE_test": round(mae_te, 2),
        "RMSE_test": round(rmse_te, 2),
        "R2_test": round(r2_te, 4),
    })

results_df = pd.DataFrame(results).sort_values("RMSE_test")
print("\nBaseline comparison (metrics on original shares scale):")
display(results_df)

# --- 3.5. Keep best model for downstream tuning
best_name = results_df.iloc[0]["model"]
best_model = models[best_name]
print(f"\nSelected baseline for tuning: {best_name}")

# Persist the feature list used at training time (helps reproducibility)
feature_list = pd.Series(X.columns, name="feature")
feature_list.to_csv("/content/drive/MyDrive/Colab Notebooks/Maestría MNA - IA Aplicada/TR5 - MLOPS/Fase 1/final_dataset/features_used.csv", index=False)
print("Saved feature list to Drive.")

Final feature count: 46

Baseline comparison (metrics on original shares scale):


,model,features,MAE_train,RMSE_train,R2_train,MAE_test,RMSE_test,R2_test
3,RandomForest,46,1896.50,4066.80,0.0022,1878.84,4028.64,0.0066
2,KNN(k=15),46,1943.93,4153.93,-0.0410,1924.58,4111.83,-0.0348
0,Linear,46,2370.33,61616.47,-228.0446,1911.31,4114.95,-0.0364
1,Ridge(α=5),46,2370.11,61584.80,-227.8093,1911.31,4114.96,-0.0364



Selected baseline for tuning: RandomForest
Saved feature list to Drive.


## Model Performance Summary and Next Steps

The baseline model comparison revealed that predicting article popularity (shares) remains an inherently noisy and weakly correlated task.
Despite extensive feature cleaning and normalization, most models achieved low explanatory power (R² ≈ 0.00–0.01), confirming the dataset’s high variance and heavy-tailed target distribution.

Among all baselines, the Random Forest Regressor delivered the best generalization performance with the lowest MAE and RMSE.
However, the differences between models were modest, indicating that the underlying signal-to-noise ratio is very low and that future gains will rely on non-linear or ensemble methods with better variance handling.

To strengthen the baseline evaluation, we will now test two additional models often used in real-world tabular data:

Decision Tree Regressor, a single-tree baseline to visualize and interpret structural patterns.

XGBoost Regressor, a gradient-boosted ensemble capable of capturing complex, non-linear relationships and handling skewed data distributions efficiently.

These additions will allow us to validate whether gradient boosting provides measurable performance improvements over bagging-based methods like Random Forest, and to select the most promising candidate for hyperparameter tuning and model explainability in later phases.

In [ ]:
# ================================================================
# Extra models: Decision Tree and XGBoost on log(1 + shares)
# ================================================================

from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor

# Try to import XGBoost; install if missing (works in Colab)
try:
    from xgboost import XGBRegressor
except Exception as e:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "xgboost"])
    from xgboost import XGBRegressor

# --- Safety checks: we assume prior cells defined these objects
_required = ["X_train", "X_test", "y_train", "y_test", "eval_regression", "X", "y_train_log", "y_test_log"]
missing = [v for v in _required if v not in globals()]
if missing:
    raise RuntimeError(f"Missing variables from previous cells: {missing}. "
                       "Run the setup, feature selection, and baseline blocks first.")

# --- KFold protocol (consistent with prior baselines)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

extra_models = {
    "DecisionTree": DecisionTreeRegressor(
        max_depth=None, min_samples_leaf=3, random_state=42
    ),
    "XGBoost": XGBRegressor(
        n_estimators=250,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        objective="reg:squarederror",
        n_jobs=-1,
        random_state=42,
        verbosity=0,
    ),
}

extra_results = []
for name, model in extra_models.items():
    # Fit on train (log scale)
    model.fit(X_train, y_train_log)

    # CV predictions on train folds (log scale)
    y_train_log_cv = cross_val_predict(model, X_train, y_train_log, cv=kf, n_jobs=-1, method="predict")
    mae_tr = mean_absolute_error(y_train, np.expm1(y_train_log_cv))
    rmse_tr = np.sqrt(mean_squared_error(y_train, np.expm1(y_train_log_cv)))
    r2_tr   = r2_score(y_train, np.expm1(y_train_log_cv))

    # Holdout test (log scale)
    y_test_log_pred = model.predict(X_test)
    mae_te = mean_absolute_error(y_test, np.expm1(y_test_log_pred))
    rmse_te = np.sqrt(mean_squared_error(y_test, np.expm1(y_test_log_pred)))
    r2_te   = r2_score(y_test, np.expm1(y_test_log_pred))

    extra_results.append({
        "model": name,
        "features": X.shape[1],
        "MAE_train": round(mae_tr, 2),
        "RMSE_train": round(rmse_tr, 2),
        "R2_train": round(r2_tr, 4),
        "MAE_test": round(mae_te, 2),
        "RMSE_test": round(rmse_te, 2),
        "R2_test": round(r2_te, 4),
    })

extra_df = pd.DataFrame(extra_results).sort_values("RMSE_test")
print("\nAdditional models (metrics on original shares scale):")
display(extra_df)

# Optionally, concatenate with previous results_df if it exists
if "results_df" in globals():
    all_models_df = pd.concat([results_df, extra_df], ignore_index=True).sort_values("RMSE_test")
    print("\nAll models summary:")
    display(all_models_df)
else:
    all_models_df = extra_df.copy()

# Pick the best overall for potential tuning
best_row = all_models_df.iloc[0]
print(f"\nCurrent best candidate: {best_row['model']}  |  RMSE_test={best_row['RMSE_test']}, MAE_test={best_row['MAE_test']}, R2_test={best_row['R2_test']}")


Additional models (metrics on original shares scale):


,model,features,MAE_train,RMSE_train,R2_train,MAE_test,RMSE_test,R2_test
1,XGBoost,46,1881.58,4058.26,0.0064,1851.43,4018.48,0.0116
0,DecisionTree,46,2759.30,5043.86,-0.5348,2712.27,4930.57,-0.4880



All models summary:


,model,features,MAE_train,RMSE_train,R2_train,MAE_test,RMSE_test,R2_test
4,XGBoost,46,1881.58,4058.26,0.0064,1851.43,4018.48,0.0116
0,RandomForest,46,1896.50,4066.80,0.0022,1878.84,4028.64,0.0066
1,KNN(k=15),46,1943.93,4153.93,-0.0410,1924.58,4111.83,-0.0348
2,Linear,46,2370.33,61616.47,-228.0446,1911.31,4114.95,-0.0364
3,Ridge(α=5),46,2370.11,61584.80,-227.8093,1911.31,4114.96,-0.0364
5,DecisionTree,46,2759.30,5043.86,-0.5348,2712.27,4930.57,-0.4880



Current best candidate: XGBoost  |  RMSE_test=4018.48, MAE_test=1851.43, R2_test=0.0116


### Final Summary and Conclusions – Phase 1: Model Construction, Tuning, and Evaluation

Throughout this notebook, we trained, compared, and evaluated multiple regression models to predict the number of social media shares for online news articles.  
The goal was to assess the predictive potential of the cleaned dataset and identify the model that best generalizes the target distribution.

---

**Main Results**

| Model | MAE_test | RMSE_test | R²_test |
|:------|----------:|-----------:|---------:|
| **XGBoost** | **1851.43** | **4018.48** | **0.0116** |
| Random Forest | 1878.84 | 4028.64 | 0.0066 |
| KNN (k=15) | 1924.58 | 4111.83 | -0.0348 |
| Ridge Regression | 1911.31 | 4114.96 | -0.0364 |
| Linear Regression | 1911.31 | 4114.95 | -0.0364 |
| Decision Tree | 2712.27 | 4930.57 | -0.4880 |

---

**Interpretation**

- The **XGBoost Regressor** achieved the best overall performance, showing a modest but consistent improvement in error metrics and a positive R² value.  
- The **Random Forest** followed closely, confirming that ensemble methods are the most suitable for this dataset’s complexity and noise.  
- Simpler models such as **Linear** or **Ridge Regression** failed to capture non-linear relationships, while the **Decision Tree** suffered from overfitting.  
- Overall, the results confirm that predicting virality remains a challenging task: the *“shares”* variable is highly skewed and noisy, making perfect predictability unattainable.

---

**Conclusion**

This notebook successfully concludes the **Modeling and Evaluation phase** of *Phase 1 (MLOps Project)*.  
The team implemented a reproducible workflow covering:

- Data cleaning, preprocessing, and scaling.  
- Feature selection and removal of redundant variables.  
- Training and cross-validation of multiple regression baselines.  
- Model comparison using MAE, RMSE, and R² on the original target scale.  

The **XGBoost Regressor** is selected as the **final baseline model** for downstream tuning and explainability in later stages of the MLOps pipeline.  
Future improvements may include **hyperparameter optimization**, **target transformation experiments**, and **ensemble blending** to enhance predictive stability.

---